CAMS

- mean, median, std, min, max for CO values by year and year-month

- maps for both results


Yearly

In [1]:
import xarray as xr
import pandas as pd
import os
import numpy as np
from collections import defaultdict

# Define the folder containing NetCDF files
folder_path = r"E:\IPMA\CAMS\chem_multlvl\5compile_2003_2024"

# Get a sorted list of all NetCDF files in the folder
file_list = sorted([f for f in os.listdir(folder_path) if f.endswith(".nc")])

# Initialize an empty list to store yearly DataFrames
df_list = []

# Process each file (each corresponding to a year)
for file in file_list:
    file_path = os.path.join(folder_path, file)
    
    ds = xr.open_dataset(file_path)  # Load without specifying chunks
    ds = ds.chunk({'time': 500})  # Rechunk after loading 

    # Extract the year from the 'time' dimension
    year = ds['time'].dt.year.values[0]  # Extract the year from the first time step
    
    # Get the total co data as a numpy array
    co_data = ds['co'].values  # Numpy array (all time steps)

    # Compute statistics using numpy's nan functions
    mean = np.nanmean(co_data, axis=0)
    median = np.nanmedian(co_data, axis=0)
    std = np.nanstd(co_data, axis=0)
    max_ = np.nanmax(co_data, axis=0)
    min_ = np.nanmin(co_data, axis=0)
    total = np.nansum(co_data, axis=0)
    
    # Combine results into a new dataset
    stats = xr.Dataset({
        'Mean': (['latitude', 'longitude'], mean),
        'Median': (['latitude', 'longitude'], median),
        'Std': (['latitude', 'longitude'], std),
        'Max': (['latitude', 'longitude'], max_),
        'Min': (['latitude', 'longitude'], min_),
        'Total': (['latitude', 'longitude'], total)
    }, coords={'latitude': ds['latitude'], 'longitude': ds['longitude']})

    # Convert to DataFrame and restructure
    stats_df = stats.to_dataframe().reset_index()
    stats_df['Year'] = year  # Add the year directly from time
    stats_df = stats_df.set_index(['Year', 'latitude', 'longitude'])
    
    # Append to list
    df_list.append(stats_df)

# Concatenate all years into a single DataFrame
df_final = pd.concat(df_list)

# Display the DataFrame
print(df_final)


                             Mean    Median       Std       Max       Min  \
Year latitude longitude                                                     
2003 66.0     -12.00     0.173757  0.170128  0.028549  0.331622  0.111273   
              -11.25     0.173908  0.170235  0.028747  0.328843  0.111275   
              -10.50     0.174010  0.170327  0.028977  0.333910  0.111241   
              -9.75      0.174145  0.170403  0.029205  0.337185  0.111250   
              -9.00      0.174192  0.170949  0.029353  0.339966  0.111290   
...                           ...       ...       ...       ...       ...   
2024 34.5      33.00     0.148935  0.144641  0.030172  0.509304  0.103343   
               33.75     0.152446  0.145942  0.037151  0.539950  0.103999   
               34.50     0.163396  0.149515  0.049014  0.536881  0.102802   
               35.25     0.208694  0.169035  0.097306  0.851748  0.112220   
               36.00     0.357748  0.247551  0.266669  1.932508  0.118107   

In [2]:
# Convert the pandas DataFrame back to an xarray Dataset
df_final_xr = df_final.reset_index().set_index(['Year', 'latitude', 'longitude'])
df_final_xr = df_final_xr.to_xarray()

# Save the xarray Dataset to a NetCDF file
output_file_path = r"E:\IPMA\CAMS\chem_multlvl\yearly_co_stats.nc"
df_final_xr.to_netcdf(output_file_path)

print(f"Dataset saved to: {output_file_path}")


Dataset saved to: E:\IPMA\CAMS\chem_multlvl\yearly_co_stats.nc


In [11]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# File path
file_path = r"E:\IPMA\CAMS\chem_multlvl\yearly_co_stats.nc"
output_folder = r"E:\IPMA\CAMS\chem_multlvl\maps_year\co"
os.makedirs(output_folder, exist_ok=True)

# Load dataset
ds = xr.open_dataset(file_path)

# Variable to plot
var = 'Mean'
title = 'Mean Concentration CO (mg/m³)'

# Global color scale limits
vmin = ds[var].min().item()
vmax = ds[var].max().item()

# Colormap
cmap = 'Blues'

# Projection (assume data is in PlateCarree, common for lat/lon grids)
data_crs = ccrs.PlateCarree()

# Loop through years
for year in ds['Year'].values:
    fig, ax = plt.subplots(figsize=(8, 6),
                           subplot_kw={'projection': ccrs.PlateCarree()})

    # Select data for current year
    data = ds[var].sel(Year=year)

    # Plot the data
    im = ax.pcolormesh(ds['longitude'], ds['latitude'], data,
                       cmap=cmap, vmin=vmin, vmax=vmax,
                       shading='auto', transform=data_crs)

    # Add features (land, ocean, borders)
    ax.coastlines(resolution='10m')
    ax.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.add_feature(cfeature.OCEAN, facecolor='lightblue')
    ax.set_extent([-12, 36, 34, 66], crs=data_crs)  # Adjust to the working area

    # Gridlines (bottom and left only)
    gl = ax.gridlines(draw_labels=True, linewidth=0)
    gl.top_labels = False
    gl.right_labels = False
    gl.bottom_labels = True
    gl.left_labels = True

    # Title and colorbar
    ax.set_title(f'{title} - {year}', fontsize=12)

    # Create a colorbar with more space
    cbar = fig.colorbar(im, ax=ax, orientation='horizontal', label='mg/m³', shrink=0.7, pad=0.1)
    
    # Adjust the colorbar's position
    cbar.ax.yaxis.set_ticks_position('right')  # Put colorbar ticks on the right side

    # Save the plot
    plt.tight_layout()
    plt.savefig(os.path.join(output_folder, f"mean_concentration_co_{year}.png"), dpi=150)
    plt.close(fig)

print("✅ Mean temperature maps with adjusted colorbar spacing generated!")


✅ Mean temperature maps with adjusted colorbar spacing generated!


Monthly

In [4]:
import xarray as xr
import pandas as pd
import os
import numpy as np

# Define the folder containing NetCDF files
folder_path = r"E:\IPMA\CAMS\chem_multlvl\4conversion_2003_2024"

# Get a sorted list of all NetCDF files in the folder
file_list = sorted([f for f in os.listdir(folder_path) if f.endswith(".nc")])

# Initialize an empty list to store monthly DataFrames
df_list = []

# Process each file (each corresponding to a year)
for file in file_list:
    file_path = os.path.join(folder_path, file)
    
    ds = xr.open_dataset(file_path)  # Load without specifying chunks
    ds = ds.chunk({'time': 500})  # Rechunk after loading 
    
    # Extract year and month from the 'time' dimension
    ds.coords['year'] = ds['time'].dt.year
    ds.coords['month'] = ds['time'].dt.month
    
    # Get the total co data as a numpy array (time, latitude, longitude)
    co_data = ds['co'].values
    
    # Calculate statistics using numpy's nan functions along the 'time' dimension
    mean = np.nanmean(co_data, axis=0)
    median = np.nanmedian(co_data, axis=0)
    std = np.nanstd(co_data, axis=0)
    max_ = np.nanmax(co_data, axis=0)
    min_ = np.nanmin(co_data, axis=0)
    total = np.nansum(co_data, axis=0)
    
    # Create a new xarray Dataset for the statistics
    stats = xr.Dataset({
        'Mean': (['latitude', 'longitude'], mean),
        'Median': (['latitude', 'longitude'], median),
        'Std': (['latitude', 'longitude'], std),
        'Max': (['latitude', 'longitude'], max_),
        'Min': (['latitude', 'longitude'], min_),
        'Total': (['latitude', 'longitude'], total)
    }, coords={'latitude': ds['latitude'], 'longitude': ds['longitude']})
    
    # Convert to DataFrame and restructure
    stats_df = stats.to_dataframe().reset_index()
    stats_df['Year'] = ds['year'].values[0]  # Add the year directly from time
    stats_df['Month'] = ds['month'].values[0]  # Add the month directly from time
    stats_df = stats_df.set_index(['Year', 'Month', 'latitude', 'longitude'])
    
    # Append to list
    df_list.append(stats_df)

# Concatenate all months into a single DataFrame
df_final = pd.concat(df_list)

# Display the DataFrame
print(df_final)
df_final


                                   Mean    Median       Std       Max  \
Year Month latitude longitude                                           
2003 1     66.0     -12.00     0.185194  0.184976  0.008918  0.212062   
                    -11.25     0.184983  0.184758  0.008652  0.211549   
                    -10.50     0.184891  0.184202  0.008555  0.209784   
                    -9.75      0.184965  0.183907  0.008455  0.209917   
                    -9.00      0.184841  0.184554  0.008277  0.210203   
...                                 ...       ...       ...       ...   
2024 11    34.5      33.00     0.147462  0.143803  0.023223  0.213272   
                     33.75     0.150223  0.142880  0.027316  0.237354   
                     34.50     0.162756  0.148896  0.038846  0.314485   
                     35.25     0.197594  0.171459  0.058561  0.415824   
                     36.00     0.436365  0.331662  0.259935  1.226923   

                                    Min       Tota

Mean    Median       Std       Max  \
Year Month latitude longitude                                           
2003 1     66.0     -12.00     0.185194  0.184976  0.008918  0.212062   
                    -11.25     0.184983  0.184758  0.008652  0.211549   
                    -10.50     0.184891  0.184202  0.008555  0.209784   
                    -9.75      0.184965  0.183907  0.008455  0.209917   
                    -9.00      0.184841  0.184554  0.008277  0.210203   
...                                 ...       ...       ...       ...   
2024 11    34.5      33.00     0.147462  0.143803  0.023223  0.213272   
                     33.75     0.150223  0.142880  0.027316  0.237354   
                     34.50     0.162756  0.148896  0.038846  0.314485   
                     35.25     0.197594  0.171459  0.058561  0.415824   
                     36.00     0.436365  0.331662  0.259935  1.226923   

                                    Min       Total  
Year Month latitude longitude                        
2003 1     66.0     -12.00     0.165758   45.928047  
                    -11.25     0.165200   45.875748  
                    -10.50     0.164725   45.852925  
                    -9.75      0.165749   45.871278  
                    -9.00      0.165201   45.840513  
...                                 ...         ...  
2024 11    34.5      33.00     0.110848   35.390980  
                     33.75     0.110284   36.053516  
                     34.50     0.113209   39.061520  
                     35.25     0.123129   47.422596  
                     36.00     0.140343  104.727623  

[735085 rows x 6 columns]

In [5]:
# Convert the pandas DataFrame back to an xarray Dataset
df_final_xr = df_final.reset_index().set_index(['Year', 'Month', 'latitude', 'longitude'])
df_final_xr = df_final_xr.to_xarray()

# Save the xarray Dataset to a NetCDF file
output_file_path = r"E:\IPMA\CAMS\chem_multlvl\monthly_co_stats.nc"
df_final_xr.to_netcdf(output_file_path)

print(f"Dataset saved to: {output_file_path}")

df_final_xr


Dataset saved to: E:\IPMA\CAMS\chem_multlvl\monthly_co_stats.nc


<xarray.Dataset> Size: 35MB
Dimensions:    (Year: 22, Month: 12, latitude: 43, longitude: 65)
Coordinates:
  * Year       (Year) int64 176B 2003 2004 2005 2006 ... 2021 2022 2023 2024
  * Month      (Month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
  * latitude   (latitude) float64 344B 34.5 35.25 36.0 36.75 ... 64.5 65.25 66.0
  * longitude  (longitude) float64 520B -12.0 -11.25 -10.5 ... 34.5 35.25 36.0
Data variables:
    Mean       (Year, Month, latitude, longitude) float64 6MB 0.1774 ... nan
    Median     (Year, Month, latitude, longitude) float64 6MB 0.1773 ... nan
    Std        (Year, Month, latitude, longitude) float64 6MB 0.02697 ... nan
    Max        (Year, Month, latitude, longitude) float64 6MB 0.2482 ... nan
    Min        (Year, Month, latitude, longitude) float64 6MB 0.1293 ... nan
    Total      (Year, Month, latitude, longitude) float64 6MB 44.0 44.26 ... nan

In [9]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# File path
file_path = r"E:\IPMA\CAMS\chem_multlvl\monthly_co_stats.nc"
output_folder = r"E:\IPMA\CAMS\chem_multlvl\maps_month\co"
os.makedirs(output_folder, exist_ok=True)

# Load dataset
ds = xr.open_dataset(file_path)

# Variable to plot
var = 'Mean'
title = 'Mean Concentration CO (mg/m³)'

# Global color scale limits
vmin = ds[var].min().item()
vmax = ds[var].max().item()

# Colormap
cmap = 'Blues'

# Coordinates
lat = ds['latitude']
lon = ds['longitude']

# Loop through all years and months
for year in ds['Year'].values:
    for month in ds['Month'].values:
        data = ds[var].sel(Year=year, Month=month)

        # Skip empty or all-NaN values
        if np.isnan(data).all():
            continue

        # Plotting
        fig = plt.figure(figsize=(7, 6))
        ax = plt.axes(projection=ccrs.PlateCarree())
        ax.set_extent([-12, 36, 34, 66], crs=ccrs.PlateCarree())

        im = ax.pcolormesh(lon, lat, data, cmap=cmap, vmin=vmin, vmax=vmax,
                           shading='auto', transform=ccrs.PlateCarree())

        ax.set_title(f'{title} - {year} / {month:02d}', fontsize=12)
        ax.coastlines(resolution='10m', linewidth=0.8)
        ax.add_feature(cfeature.BORDERS, linewidth=0.5)
        ax.add_feature(cfeature.LAND, edgecolor='black', facecolor='none')

        # Gridlines (bottom and left only)
        gl = ax.gridlines(draw_labels=True, linewidth=0)
        gl.top_labels = False
        gl.right_labels = False
        gl.bottom_labels = True
        gl.left_labels = True

        # Colorbar
        cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.7, pad=0.1)
        cbar.set_label('mg/m³')
        cbar.ax.tick_params(labelsize=8)

        # Save figure
        plt.tight_layout()
        filename = f"mean_concentration_co_{year}{month:02d}.png"
        plt.savefig(os.path.join(output_folder, filename), dpi=150)
        plt.close(fig)

print("✅ Monthly temperature maps generated for all years and months!")


✅ Monthly temperature maps generated for all years and months!


In [10]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# File path
file_path = r"E:\IPMA\CAMS\chem_multlvl\monthly_co_stats.nc"
output_folder = r"E:\IPMA\CAMS\chem_multlvl\maps_month\co"
os.makedirs(output_folder, exist_ok=True)

# Load dataset
ds = xr.open_dataset(file_path)

# Variable to plot
var = 'Mean'
title = 'Mean Concentration CO (mg/m³)'

# Global color scale limits
vmin = ds[var].min().item()
vmax = ds[var].max().item()

# Colormap
cmap = 'Blues'

# Coordinates
lat = ds['latitude']
lon = ds['longitude']

# Loop through all years
for year in ds['Year'].values:
    fig, axs = plt.subplots(3, 4, figsize=(15, 10),
                            subplot_kw={'projection': ccrs.PlateCarree()})
    fig.suptitle(f'{title} - {year}', fontsize=16)

    for i, month in enumerate(ds['Month'].values):
        row, col = divmod(i, 4)
        ax = axs[row, col]

        # Extract data
        data = ds[var].sel(Year=year, Month=month)

        # Skip if no data
        if np.isnan(data).all():
            ax.set_visible(False)
            continue

        ax.set_extent([-12, 36, 34, 66], crs=ccrs.PlateCarree())

        im = ax.pcolormesh(lon, lat, data, cmap=cmap, vmin=vmin, vmax=vmax,
                           shading='auto', transform=ccrs.PlateCarree())

        ax.set_title(f'Month {month:02d}', fontsize=10)
        ax.coastlines(resolution='10m', linewidth=0.6)
        ax.add_feature(cfeature.BORDERS, linewidth=0.4)
        ax.add_feature(cfeature.LAND, edgecolor='black', facecolor='none')

        # Gridlines (bottom and left only)
        gl = ax.gridlines(draw_labels=True, linewidth=0)
        gl.top_labels = False
        gl.right_labels = False
        gl.bottom_labels = row == 2
        gl.left_labels = col == 0

    # Add a single colorbar for all subplots
    cbar_ax = fig.add_axes([0.92, 0.25, 0.015, 0.5])
    cbar = fig.colorbar(im, cax=cbar_ax)
    cbar.set_label('mg/m³')

    # Save figure
    plt.tight_layout(rect=[0, 0, 0.9, 0.95])
    filename = f"mean_concentration_co_year_{year}.png"
    plt.savefig(os.path.join(output_folder, filename), dpi=150)
    plt.close(fig)

print("✅ Yearly temperature grids generated (3x4 months per year)!")


C:\Users\sofia\AppData\Local\Temp\ipykernel_11556\1569337451.py:72: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.9, 0.95])
C:\Users\sofia\AppData\Local\Temp\ipykernel_11556\1569337451.py:72: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.9, 0.95])
C:\Users\sofia\AppData\Local\Temp\ipykernel_11556\1569337451.py:72: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.9, 0.95])
C:\Users\sofia\AppData\Local\Temp\ipykernel_11556\1569337451.py:72: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.9, 0.95])
C:\Users\sofia\AppData\Local\Temp\ipykernel_11556\1569337451.py:72: UserWarning: This figure includes Ax

✅ Yearly temperature grids generated (3x4 months per year)!


Daily

In [3]:
import xarray as xr
import pandas as pd
import os
import numpy as np

# Define the folder containing NetCDF files
folder_path = r"E:\IPMA\CAMS\chem_multlvl\4conversion_2003_2024"

# Get a sorted list of all NetCDF files in the folder
file_list = sorted([f for f in os.listdir(folder_path) if f.endswith(".nc")])

# Initialize an empty list to store monthly DataFrames
df_list = []

# Process each file (each corresponding to a year)
for file in file_list:
    file_path = os.path.join(folder_path, file)
    
    ds = xr.open_dataset(file_path)  # Load without specifying chunks
    ds = ds.chunk({'time': 500})  # Rechunk after loading 

    # Group by day
    daily_grouped = ds.groupby('time.date')  # Group by each unique date
    
    # Calculate statistics using numpy's nan functions along the 'time' dimension
    for date, daily_data in daily_grouped:
        co_data = daily_data['co'].values
        
        mean = np.nanmean(co_data, axis=0)
        median = np.nanmedian(co_data, axis=0)
        std = np.nanstd(co_data, axis=0)
        max_ = np.nanmax(co_data, axis=0)
        min_ = np.nanmin(co_data, axis=0)
        total = np.nansum(co_data, axis=0)
    
        # Create a new xarray Dataset for the statistics
        stats = xr.Dataset({
            'Mean': (['latitude', 'longitude'], mean),
            'Median': (['latitude', 'longitude'], median),
            'Std': (['latitude', 'longitude'], std),
            'Max': (['latitude', 'longitude'], max_),
            'Min': (['latitude', 'longitude'], min_),
            'Total': (['latitude', 'longitude'], total)
        }, coords={'latitude': ds['latitude'], 'longitude': ds['longitude']})

        # Convert to DataFrame and restructure
        stats_df = stats.to_dataframe().reset_index()
        date_pd = pd.to_datetime(str(date))
        stats_df['Year'] = date_pd.year
        stats_df['Month'] = date_pd.month
        stats_df['Day'] = date_pd.day
        stats_df = stats_df.set_index(['Year', 'Month', 'Day', 'latitude', 'longitude'])
    
        # Append to list
        df_list.append(stats_df)
        
# Concatenate all months into a single DataFrame
df_final = pd.concat(df_list)

# Display the DataFrame
print(df_final)
df_final


                                       Mean    Median       Std       Max  \
Year Month Day latitude longitude                                           
2003 1     1   66.0     -12.00     0.170383  0.168498  0.005059  0.182064   
                        -11.25     0.171718  0.169803  0.006490  0.184994   
                        -10.50     0.172361  0.169036  0.007715  0.185087   
                        -9.75      0.173009  0.168084  0.008068  0.185175   
                        -9.00      0.174016  0.169506  0.008743  0.185339   
...                                     ...       ...       ...       ...   
2024 11    30  34.5      33.00     0.170014  0.169243  0.008629  0.185118   
                         33.75     0.185985  0.188462  0.010260  0.196828   
                         34.50     0.199901  0.203669  0.010703  0.211266   
                         35.25     0.222107  0.222798  0.012153  0.237516   
                         36.00     0.500429  0.606433  0.186156  0.677293   

Mean    Median       Std       Max  \
Year Month Day latitude longitude                                           
2003 1     1   66.0     -12.00     0.170383  0.168498  0.005059  0.182064   
                        -11.25     0.171718  0.169803  0.006490  0.184994   
                        -10.50     0.172361  0.169036  0.007715  0.185087   
                        -9.75      0.173009  0.168084  0.008068  0.185175   
                        -9.00      0.174016  0.169506  0.008743  0.185339   
...                                     ...       ...       ...       ...   
2024 11    30  34.5      33.00     0.170014  0.169243  0.008629  0.185118   
                         33.75     0.185985  0.188462  0.010260  0.196828   
                         34.50     0.199901  0.203669  0.010703  0.211266   
                         35.25     0.222107  0.222798  0.012153  0.237516   
                         36.00     0.500429  0.606433  0.186156  0.677293   

                                        Min     Total  
Year Month Day latitude longitude                      
2003 1     1   66.0     -12.00     0.165758  1.363066  
                        -11.25     0.165200  1.373744  
                        -10.50     0.164725  1.378891  
                        -9.75      0.165749  1.384075  
                        -9.00      0.165201  1.392129  
...                                     ...       ...  
2024 11    30  34.5      33.00     0.156566  1.360116  
                         33.75     0.164084  1.487881  
                         34.50     0.175673  1.599211  
                         35.25     0.196285  1.776855  
                         36.00     0.226072  4.003436  

[22373975 rows x 6 columns]

In [4]:
# Convert the pandas DataFrame back to an xarray Dataset
df_final_xr = df_final.reset_index().set_index(['Year', 'Month', 'Day', 'latitude', 'longitude'])
df_final_xr = df_final_xr.to_xarray()

# Save the xarray Dataset to a NetCDF file
output_file_path = r"E:\IPMA\CAMS\chem_multlvl\daily_co_stats.nc"
df_final_xr.to_netcdf(output_file_path)

print(f"Dataset saved to: {output_file_path}")

df_final_xr


Dataset saved to: E:\IPMA\CAMS\chem_multlvl\daily_co_stats.nc


<xarray.Dataset> Size: 1GB
Dimensions:    (Year: 22, Month: 12, Day: 31, latitude: 43, longitude: 65)
Coordinates:
  * Year       (Year) int64 176B 2003 2004 2005 2006 ... 2021 2022 2023 2024
  * Month      (Month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
  * Day        (Day) int64 248B 1 2 3 4 5 6 7 8 9 ... 23 24 25 26 27 28 29 30 31
  * latitude   (latitude) float64 344B 34.5 35.25 36.0 36.75 ... 64.5 65.25 66.0
  * longitude  (longitude) float64 520B -12.0 -11.25 -10.5 ... 34.5 35.25 36.0
Data variables:
    Mean       (Year, Month, Day, latitude, longitude) float64 183MB 0.1544 ....
    Median     (Year, Month, Day, latitude, longitude) float64 183MB 0.1543 ....
    Std        (Year, Month, Day, latitude, longitude) float64 183MB 0.001885...
    Max        (Year, Month, Day, latitude, longitude) float64 183MB 0.1567 ....
    Min        (Year, Month, Day, latitude, longitude) float64 183MB 0.1515 ....
    Total      (Year, Month, Day, latitude, longitude) float64 183MB 1.235 .....